In [1]:
from sklearn import svm
import random
import numpy as np
import matplotlib.pyplot as plt
from smt.surrogate_models import KRG
from cec2017.functions import all_functions
from cec2017.functions import f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f15, f16

In [2]:
def create_pop(size, n, min_val, max_val):
    population = np.zeros((size, n))
    for i in range(size):
        population[i] = np.random.uniform(min_val, max_val, size=n)
    return population


def mutation(sort_index_fit, population, F, pop_size, j):
    a = j
    b = j
    c = j
    while a == j:
        a = sort_index_fit[np.random.randint(0, pop_size*0.2)]
    while b == j or b == a:
        b = np.random.randint(0, pop_size)
    while c == j or c == b or c == a:
        c = np.random.randint(0, pop_size)
    #print("mut", "a:", population[a], "b:", population[b], "c:", population[c], "j:", population[j])
    #print(population)
    mutated_vector = population[j] + F * (population[a] - population[j]) + F * (population[b] - population[c])
    return mutated_vector


def crossover(target, mutant, crossover_prob):
    crossed = np.zeros(n)
    indx = np.random.randint(0, n)
    for i in range(n):
        if random.random() < crossover_prob or i == indx:
            crossed[i] = mutant[i]
        else:
            crossed[i] = target[i]
    return crossed


def func(x):
    return x[0] ** 2 + x[1] ** 2

def rosenbrock(x):
    x1, x2 = x[0], x[1]
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2

In [40]:
def differential_evolution(mode, objective_func, pop_size, n_params, min_val, max_val, num_calculate):
    calc = pop_size
    iter = 0
    population = create_pop(pop_size, n_params, min_val, max_val)
    fit_pop = objective_func(population)
    count_calculate = pop_size
    sort_index_fit = np.argsort(fit_pop.flatten())
    H = 5
    MF = np.ones(H) * 0.5
    MCr = np.ones(H) * 0.5
    Sf = np.zeros(pop_size)
    Scr = np.zeros(pop_size)
    d_f = np.zeros(pop_size)
    if mode == 1:
        surr = 0
        xt = np.copy(population)
        print(xt.shape)
        yt = np.copy(fit_pop).reshape(pop_size, 1)
        print(yt.shape)
        reg = svm.SVR()
        reg.fit(xt, yt)

    while calc < num_calculate:
        count = 0
        iter += 1

        for idx, ind in enumerate(population):
            target = ind
            r = np.random.randint(0, H)
            F = -1
            while F < 0 or F > 1:
                F = np.random.standard_cauchy() * 0.1 + MF[r]

            crossover_prob = -1
            while crossover_prob < 0 or crossover_prob > 1:
                crossover_prob = np.random.normal(MCr[r], 0.1)

            while True:
                index = random.randint(0, pop_size - 1)
                if index != idx:
                    break

            mutant = mutation(sort_index_fit, population, F, pop_size, idx)
            crossed = crossover(target, mutant, crossover_prob)
            corrected_vector = np.array([])
            for i in range(len(crossed)):
                if crossed[i] < min_val:
                    corrected_vector = np.append(corrected_vector, (target[i] + min_val) / 2)
                elif crossed[i] > max_val:
                    corrected_vector = np.append(corrected_vector, (target[i] + max_val) / 2)
                else:
                    corrected_vector = np.append(corrected_vector, crossed[i])

            if mode == 1:
                if iter % 2 == 0:
                    fit_new = objective_func(corrected_vector.reshape((1, n)))
                    calc += 1
                    xt = np.vstack((xt, corrected_vector))
                    yt = np.vstack((yt, fit_new))
                else:
                    surr += 1
                    fit_new = reg.predict(corrected_vector.reshape(1, -1))

            else:
                fit_new = objective_func(corrected_vector.reshape((1, n)))
                calc += 1

            if fit_new < fit_pop[idx]:
                Sf[count] = F
                Scr[count] = crossover_prob
                d_f[count] = fit_pop[idx] - fit_new
                population[idx] = corrected_vector
                fit_pop[idx] = fit_new
                count += 1

            weights = np.zeros(pop_size)
            sum_d_f = np.sum(d_f[:count])
            if sum_d_f > 0:
                for c in range(count):
                    weights[c] = d_f[c] / sum_d_f

                mean_1 = 0
                mean_2 = 0

                for c in range(count):
                    mean_1 += weights[c] * Sf[c] * Sf[c]
                    mean_2 += weights[c] * Sf[c]

                MF[iter % H] = mean_1 / mean_2
                mean_1 = 0
                mean_2 = 0

                for c in range(count):
                    mean_1 += weights[c] * Scr[c] * Scr[c]
                    mean_2 += weights[c] * Scr[c]

                MCr[iter % H] = mean_1 / mean_2


        #print("Gen:", gen)
        if mode == 1:
            print(f"Поколение {iter}. Вычисления: {calc}, Суррогата: {surr}")
            if iter % 2 == 0:
                print(f"Обучение на данных: {xt.shape}")
                
                reg.fit(xt, yt.reshape(-1, ))
                print(yt.shape)
        else:
            print(f"Поколение {iter}. Вычисления: {calc}")

    best_solution = xt[np.argmin(yt)]
    print(np.min(yt))


    return best_solution, np.min(yt)

In [41]:
size = 20
n = 10
min_val = -100
max_val = 100
modes = 2
num_calculate = 400
num_exp = 1

In [43]:
functions = [f11]
for mo in range(1, modes):
    for func in functions:
        fit_list = np.array([])
        for exp in range(num_exp):
            best_solution, fit = differential_evolution(mo, func, size, n, min_val, max_val, num_calculate)
            fit_list = np.append(fit_list, fit)
        np.savetxt(f'result_{func.__name__}_mode_{mo}.txt', fit_list)
        print(f"{func.__name__} finished!")
    #print("Лучшее решение:", best_solution)
    #print("Значение функции:", fit)

(20, 10)
(20, 1)
Поколение 1. Вычисления: 20, Суррогата: 20
Поколение 2. Вычисления: 40, Суррогата: 20
Обучение на данных: (40, 10)
(40, 1)
Поколение 3. Вычисления: 40, Суррогата: 40
Поколение 4. Вычисления: 60, Суррогата: 40
Обучение на данных: (60, 10)
(60, 1)
Поколение 5. Вычисления: 60, Суррогата: 60
Поколение 6. Вычисления: 80, Суррогата: 60
Обучение на данных: (80, 10)
(80, 1)
Поколение 7. Вычисления: 80, Суррогата: 80
Поколение 8. Вычисления: 100, Суррогата: 80
Обучение на данных: (100, 10)
(100, 1)
Поколение 9. Вычисления: 100, Суррогата: 100
Поколение 10. Вычисления: 120, Суррогата: 100
Обучение на данных: (120, 10)
(120, 1)
Поколение 11. Вычисления: 120, Суррогата: 120
Поколение 12. Вычисления: 140, Суррогата: 120
Обучение на данных: (140, 10)
(140, 1)
Поколение 13. Вычисления: 140, Суррогата: 140
Поколение 14. Вычисления: 160, Суррогата: 140
Обучение на данных: (160, 10)
(160, 1)
Поколение 15. Вычисления: 160, Суррогата: 160
Поколение 16. Вычисления: 180, Суррогата: 160
Обуч

C:\Users\jeela\anaconda3\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Поколение 20. Вычисления: 220, Суррогата: 200
Обучение на данных: (220, 10)
(220, 1)
Поколение 21. Вычисления: 220, Суррогата: 220
Поколение 22. Вычисления: 240, Суррогата: 220
Обучение на данных: (240, 10)
(240, 1)
Поколение 23. Вычисления: 240, Суррогата: 240
Поколение 24. Вычисления: 260, Суррогата: 240
Обучение на данных: (260, 10)
(260, 1)
Поколение 25. Вычисления: 260, Суррогата: 260
Поколение 26. Вычисления: 280, Суррогата: 260
Обучение на данных: (280, 10)
(280, 1)
Поколение 27. Вычисления: 280, Суррогата: 280
Поколение 28. Вычисления: 300, Суррогата: 280
Обучение на данных: (300, 10)
(300, 1)
Поколение 29. Вычисления: 300, Суррогата: 300
Поколение 30. Вычисления: 320, Суррогата: 300
Обучение на данных: (320, 10)
(320, 1)
Поколение 31. Вычисления: 320, Суррогата: 320
Поколение 32. Вычисления: 340, Суррогата: 320
Обучение на данных: (340, 10)
(340, 1)
Поколение 33. Вычисления: 340, Суррогата: 340
Поколение 34. Вычисления: 360, Суррогата: 340
Обучение на данных: (360, 10)
(360, 1

In [ ]:
print(f"Кол-во индивидов: {size}, Кол-во вычислений: {num_calculate}, n = {n}")
functions = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f15, f16]
dict_mean = {}
dict_mean['ДЭ'] = []
dict_mean['ДЭ + SVM'] = []
for func in functions:
    data1 = np.loadtxt(f'result_{func.__name__}_mode_0.txt')
    data2 = np.loadtxt(f'result_{func.__name__}_mode_1.txt')
    mean_value1 = np.mean(data1)
    mean_value2 = np.mean(data2)
    dict_mean['ДЭ'].append(mean_value1)
    dict_mean['ДЭ + SVR'].append(mean_value1)
    
    dict_mean[f'{test.__name__}'] = [mean_value1, mean_value2]

mean_df = pd.DataFrame(dict_mean, index=['ДЭ', 'ДЭ + SVR'])

print(mean_df)